<h3> <strong> 1. 실험 </strong> </h3>

<h4> <strong>[비교] (label_smoothing+evolve 모델+multi-scale -> TTA 적용 & 미적용)</strong> </h4>

**학습 명령**

* python train.py --img 1024 --batch 4 --epochs 70 --data './data/dataset.yaml' --cfg './models/yolov5x.yaml' --weights yolov5x.pt --hyp './runs/evolve/YoloV5x_Hyp_tuning/hyp_evolve.yaml' --label-smoothing 0.1 --multi-scale --name YoloV5x_multiscale --cache 
* python train.py --img 1024 --batch 4 --epochs 70 --data './data/dataset.yaml' --cfg './models/yolov5x.yaml' --weights yolov5x.pt --hyp './data/hyps/hyp.scratch-low.yaml' --label-smoothing 0.1 --multi-scale --name YoloV5x_multiscale_lowYaml --cache 
* python train.py --img 1024 --batch 4 --epochs 70 --data './data/dataset.yaml' --cfg './models/yolov5x.yaml' --weights yolov5x.pt --hyp './runs/evolve/YoloV5x_Hyp_tuning/hyp_evolve.yaml' --multi-scale --name YoloV5x_multiscale_no_LS --cache 

**테스트 명령**

* python detect.py --img 1024 --source './data/test' --weights './runs/train/YoloV5x_multiscale/weights/best.pt' --save-txt --save-conf --name YoloV5x_multiscale_no_TTA
* python detect.py --img 1024 --source './data/test' --weights './runs/train/YoloV5x_multiscale/weights/best.pt' --augment --save-txt --save-conf --name YoloV5x_multiscale_TTA
* python detect.py --img 1024 --source './data/test' --weights './runs/train/YoloV5x_multiscale/weights/best.pt' --augment --save-txt --save-conf --name YoloV5x_multiscale_TTA_IOU --iou-thres 0.3
* python detect.py --img 1024 --source './data/test' --weights './runs/train/YoloV5x_multiscale/weights/best.pt' --augment --save-txt --save-conf --name YoloV5x_multiscale_lowYaml
* python detect.py --img 1024 --source './data/test' --weights './runs/train/YoloV5x_multiscale/weights/best.pt' --augment --save-txt --save-conf --name YoloV5x_multiscale_no_LS


In [1]:
import pandas as pd
import os

df = pd.DataFrame(index=[i for i in range(4871)], columns=['PredictionString', 'image_id'])
yolo_path = './../yolov5/runs/detect/YoloV5x_multiscale_TTA_IOU/labels'
for img_index in range(4871):
    image_id = str(img_index)
    while len(image_id)<4: image_id = '0'+image_id

    PredictionString = ''
    try: 
        txt_file = open(os.path.join(yolo_path, image_id+'.txt'), 'r')
        lines = txt_file.readlines()
    
        for line in lines:
            category, yolo_x, yolo_y, yolo_w, yolo_h, pred = (line.split())
            x_min = (float(yolo_x)-float(yolo_w)/2)*1024
            y_min = (float(yolo_y)-float(yolo_h)/2)*1024
            x_max = x_min+float(yolo_w)*1024
            y_max = y_min+float(yolo_h)*1024
            PredictionString += f'{category} {pred} {x_min} {y_min} {x_max} {y_max} '
        txt_file.close()
    except FileNotFoundError:
        pass
    
    df.iloc[img_index] = [PredictionString, 'test/'+image_id+'.jpg']

df.to_csv('./YoloV5x_multiscale_TTA_IOU.csv', index=False)

<h3> <strong> 2. 결과 </strong> </h3>

(1) Multi-Scale 적용: 0.4410 / 기존 0.4623보다 0.02 정도 성능 감소함. <br>
(2) Multi-Scale 모델에 TTA 적용: 0.4938 / TTA의 성능이 좋은 것을 확인할 수 있음.<br>
(3) iou-thres 감소: 0.4944 / 0.4410에 비해 mAP 성적이 0.05정도 향상됨. <br>
(4) low.yaml 파라미터 구성 적용: (valid) 0.5511 / evolution 기법을 적용한 파라미터 조합보다 valid가 0.04정도 감소함. 따라서 제출은 하지 않음.<br>
(5) Label Smoothing 제외: 0.4938 / 0.4410에 비해 0.05 정도 성능이 향상됨. label smoothing과 multi-scale의 조합이 좋지 않은 듯 함.